In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import grad

from pickle import dump, load

from Heston_NN import Net,weights_init

In [2]:
save_net = True
#save_net = False
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
arbitrage_weight = [1e-3,1e-3]; 
l2_weight = 1e-5
arbitrage_weight = [0.0,0.0]; 
#l2_weight = 0.0


In [3]:
table = pd.read_csv('Heston_data_input')
X = table.drop(['C_price','delta'], axis=1)
y = table[['C_price']]
#X = df[['k','T','C_price']]
#y = df[['v0','rho','kappa','theta','sigma']]
T_loc = list(X.columns).index('T')
k_loc = list(X.columns).index('k') # log strike 
X

,T,k,kappa,rho,sigma,theta,v0
0,0.25,-0.2,0.001,-0.800000,0.001000,0.001,0.010000
1,0.25,-0.2,0.001,-0.622222,0.001000,0.001,0.010000
2,0.25,-0.2,0.001,-0.444444,0.001000,0.001,0.010000
3,0.25,-0.2,0.001,-0.266667,0.001000,0.001,0.010000
4,0.25,-0.2,0.001,-0.088889,0.001000,0.001,0.010000
...,...,...,...,...,...,...,...
124587,2.00,0.2,20.000,0.088889,0.178556,0.001,0.447214
124588,2.00,0.2,20.000,0.266667,0.178556,0.001,0.447214
124589,2.00,0.2,20.000,0.444444,0.178556,0.001,0.447214
124590,2.00,0.2,20.000,0.622222,0.178556,0.001,0.447214


In [4]:
table.describe()

,C_price,T,delta,k,kappa,rho,sigma,theta,v0
count,124592.000000,124592.000000,124592.000000,124592.000000,124592.000000,124592.000000,124592.000000,124592.000000,124592.000000
mean,0.123665,0.798478,0.551715,-0.000008,5.546486,-0.000011,0.094902,3.193324,0.228621
std,0.087584,0.557381,0.251790,0.127657,7.382655,0.515946,0.136277,3.561212,0.139527
min,0.001088,0.250000,-0.054625,-0.200000,0.001000,-0.800000,0.001000,0.001000,0.010000
25%,0.048538,0.250000,0.386104,-0.111111,0.001000,-0.444444,0.001000,0.001000,0.107159
50%,0.116618,0.750000,0.591939,-0.022222,0.001000,-0.088889,0.089778,1.112000,0.252896
75%,0.184500,1.000000,0.731120,0.111111,11.111556,0.444444,0.089778,6.667000,0.350055
max,0.437569,2.000000,1.072294,0.200000,20.000000,0.800000,0.800000,10.000000,0.447214


In [5]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
...     X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
...     X_train_val, y_train_val, test_size=0.5, random_state=42)


input_scaler = preprocessing.MinMaxScaler()
X_train_scaled = input_scaler.fit_transform(X_train)
X_val_scaled = input_scaler.transform(X_val)
X_test_scaled = input_scaler.transform(X_test)

#X_train_scaled = torch.FloatTensor(X_train_scaled,requires_grad=True)
#X_test_scaled = torch.FloatTensor(X_test_scaled,requires_grad=True)
X_train_scaled = torch.FloatTensor(X_train_scaled).to(device)
X_train_scaled.requires_grad = True
X_val_scaled = torch.FloatTensor(X_val_scaled).to(device)
X_val_scaled.requires_grad = True
X_test_scaled = torch.FloatTensor(X_test_scaled).to(device)
X_test_scaled.requires_grad = True

y_train = torch.FloatTensor(y_train.values).to(device)
y_val = torch.FloatTensor(y_val.values).to(device)
y_test = torch.FloatTensor(y_test.values).to(device)

if save_net:
    dump(input_scaler, open('input_scaler.pkl', 'wb'))
#    input_scaler = load(open('input_scaler.pkl', 'rb'))

In [6]:
input_scaler.scale_

array([0.57142857, 2.5       , 0.0500025 , 0.625     , 1.25156446,
       0.10001   , 2.28721158])

In [7]:
input_scaler.min_

array([-1.42857143e-01,  5.00000000e-01, -5.00025001e-05,  5.00000000e-01,
       -1.25156446e-03, -1.00010001e-04, -2.28721158e-02])

In [10]:
model = Net(num_neurons=128,device=device)
#model = Net(num_neurons=64,device=device)
model.apply(weights_init)
#model.load_state_dict(torch.load('heston_NN_intermediate')) # load model
#model.load_state_dict(torch.load('heston_NN_final')) # load model

print(model)

Net(
  (fc1): Linear(in_features=7, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (fc7): Linear(in_features=128, out_features=1, bias=True)
)


# Training

In [11]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-6, weight_decay=1e-3) # 16
#optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-3) # 16
#optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3) # 32
#optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-3) 
epochs = 100001
#epochs = 2001
#epochs = 10
loss_arr = []
val_arr =[]
freq = 500

for i in range(epochs):
    if i % freq == 0:
        print('Epoch: ',i,' ')
        show_log = True
        if save_net: 
            torch.save(model.state_dict(), "heston_NN_intermediate") # save the model
    else:
        show_log = False
 
#    loss = model.get_loss(X_train_scaled, y_train, T_loc, k_loc, criterion, arbitrage_weight, l2_weight, show_log=show_log)
#    val_loss = model.get_loss(X_val_scaled, y_val, T_loc, k_loc, criterion, arbitrage_weight, l2_weight, show_log=show_log)
    loss = model.get_loss_test(X_train_scaled, y_train, T_loc, k_loc, criterion, arbitrage_weight, l2_weight, show_log=show_log)
    val_loss = model.get_loss_test(X_val_scaled, y_val, T_loc, k_loc, criterion, arbitrage_weight, l2_weight, show_log=show_log)

    optimizer.zero_grad()
    loss.backward()
    loss_arr.append(float(loss))
    val_arr.append(float(val_loss))
    optimizer.step()
    


Epoch:  0  
Loss: 0.02911433018743992
Loss: 0.02899889647960663
Epoch:  500  
Loss: 0.026639901101589203
Loss: 0.026532890275120735
Epoch:  1000  
Loss: 0.02435345947742462
Loss: 0.024254698306322098
Epoch:  1500  
Loss: 0.022179778665304184
Loss: 0.022090280428528786
Epoch:  2000  
Loss: 0.020170090720057487
Loss: 0.02008969709277153
Epoch:  2500  
Loss: 0.018338419497013092
Loss: 0.018267249688506126
Epoch:  3000  
Loss: 0.016666904091835022
Loss: 0.016604576259851456
Epoch:  3500  
Loss: 0.015145525336265564
Loss: 0.015092069283127785
Epoch:  4000  
Loss: 0.013770725578069687
Loss: 0.013726010918617249
Epoch:  4500  
Loss: 0.012541232630610466
Loss: 0.012505298480391502
Epoch:  5000  
Loss: 0.011453254148364067
Loss: 0.011425749398767948
Epoch:  5500  
Loss: 0.010500486008822918
Loss: 0.010481026023626328
Epoch:  6000  
Loss: 0.00967701431363821
Loss: 0.009665429592132568
Epoch:  6500  
Loss: 0.00897327158600092
Loss: 0.008969082497060299
Epoch:  7000  
Loss: 0.008375789038836956
Lo

KeyboardInterrupt: 

In [ ]:
plt.title('loss')
plt.plot(loss_arr,label='train')
plt.plot(val_arr,label='validation')
plt.legend()
plt.yscale('log')

In [ ]:
if save_net:
    X_scaled = input_scaler.transform(X)
    X_scaled_tensor = torch.FloatTensor(X_scaled).to(device)
    y_tensor = torch.FloatTensor(table[['C_price']].values).to(device)
    model.train()
    for i in range(1000):
        loss = model.get_loss_test(X_scaled_tensor, y_tensor, T_loc, k_loc, criterion, arbitrage_weight, l2_weight, show_log=False)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(loss)
    torch.save(model.state_dict(), "heston_NN_final") # save the model

In [ ]:
y

In [ ]:
#m2 = Net(num_neurons=64,device=device)
#m2.load_state_dict(torch.load('heston_NN'))
#m2.load_state_dict(torch.load('heston_NN_intermediate'))
#m2.eval() # evaluation mode

# Test case

In [ ]:
loss = model.get_loss_test(X_test_scaled, y_test, T_loc, k_loc, criterion, arbitrage_weight, l2_weight, show_log=True)

# autograd

In [ ]:
x = X_train_scaled[0].clone().detach()
x.requires_grad = True
y= model.forward(x)
dydx = grad(y,x, create_graph = True)[0]

In [ ]:
if dydx[T_loc] < 0.0:
    print('nono ',torch.exp(dydx[T_loc] ))

In [ ]:
d2ydx2 = grad(dydx[T_loc],x, create_graph = True)[0]

In [ ]:
d2ydx2

In [ ]:
x = X_train_scaled.clone().detach()
x.requires_grad = True
y= model.forward(x)

In [ ]:
calendar_loss = torch.tensor(0.)
butterfly_loss = torch.tensor(0.)
for elem in X_train_scaled:
    y= model.forward(elem)
    dydx = grad(y, elem, create_graph = True)[0]
    dydT = dydx[T_loc]
    if dydT < 0.0:
        calendar_arbi_count += 1
        calendar_loss += torch.exp(-dydT[T_loc]) * 1e-1
    dydk = dydx[k_loc] # dCdk w.r.t. log-strike 
    d2ydk2 = grad(dydx[k_loc],elem, create_graph = True)[0][k_loc] # d2Cdk2 w.r.t. log-strike
    # dCdK = 1/K*dcdk
    # d2CdK2 = 1/e^2k * (d2Cdk2 - dCdk)
    # butterfly arbitrage: d2CdK2 = 1/K^2(d2Cdk2 - dCdk) > 0, d2Cdk2 > dCdk
    if d2ydk2 - dydk < 0.0: # violation of butterfly arbitrage
        butterfly_loss += torch.exp(-(d2ydk2 - dydk)) * 1e-1

In [ ]:
"""
class Net(nn.Module):

    def __init__(self,num_input=7, num_neurons=128):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(num_input, num_neurons) 
        self.fc2 = nn.Linear(num_neurons, num_neurons)
        self.fc3 = nn.Linear(num_neurons, num_neurons)
#        self.fc4 = nn.Linear(num_neurons, num_neurons)
#        self.fc5 = nn.Linear(num_neurons, num_neurons)
#        self.fc6 = nn.Linear(num_neurons, num_neurons)
        self.fc7 = nn.Linear(num_neurons, 1)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.relu(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
#        x = F.relu(self.fc3(x))
#        x = F.relu(self.fc4(x))
#        x = F.relu(self.fc5(x))
#        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        return x
    
    def get_loss(self, x, y_train, T_loc, k_loc, criterion, arbitrage_weight, l2_weight, show_log=False):
        y_hat = self.forward(x)
        loss = criterion(y_hat, y_train)
        l2_reg = torch.tensor(0.)

        # Penalization (loop over each data)
        ## calendar arbitrage
        calendar_arbi_count = 0
        calendar_loss = torch.tensor(0.)
        butterfly_loss = torch.tensor(0.)
        for elem in x:
            y= self.forward(elem)
            dydx = grad(y, elem, create_graph = True)[0]
            dydT = dydx[T_loc]
            if dydT < 0.0:
                calendar_arbi_count += 1
                calendar_loss += torch.exp(-dydT) * arbitrage_weight[0]
        loss += calendar_loss
        ## butterfly arbitrage
        butterfly_count = 0
        dydk = dydx[k_loc] # dCdk w.r.t. log-strike 
        d2ydk2 = grad(dydx[k_loc],elem, create_graph = True)[0][k_loc] # d2Cdk2 w.r.t. log-strike
        # butterfly arbitrage: d2CdK2 = 1/K^2(d2Cdk2 - dCdk) > 0, d2Cdk2 > dCdk
        # dCdK = 1/K*dcdk
        # d2CdK2 = 1/e^2k * (d2Cdk2 - dCdk)
        butter_ineq = (d2ydk2 - dydk)/torch.exp(2.0*elem[k_loc])
        if butter_ineq < 0.0: # violation of butterfly arbitrage
            butterfly_count += 1
            butterfly_loss += torch.exp(-butter_ineq) * arbitrage_weight[1]
        loss += butterfly_loss

        # regularizations
        for param in self.parameters():
            l2_reg += torch.norm(param)
            loss += l2_lambda * l2_reg
            
        if show_log:
            if calendar_arbi_count > 0: print('calendar ',i, ' ',calendar_arbi_count,' ',calendar_loss)
            if butterfly_count > 0: print('butterfly ',i, ' ',butterfly_count,' ',butterfly_loss)
            print(f'Loss: {loss}')
            
        return loss
        

def weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
#        torch.nn.init.normal_(m.weight)
#        xavier(m.weight.data)
#        xavier(m.bias.data)

"""

In [ ]:
"""
epochs = 30001
#epochs = 101
loss_arr = []
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-3)
l2_lambda = 1e-5
criterion = nn.MSELoss()


for i in range(epochs):
    y_hat = model.forward(X_train_scaled)
    loss = criterion(y_hat, y_train)
    l2_reg = torch.tensor(0.)
    
    # Penalization (loop over each data)
    ## calendar arbitrage
    calendar_arbi_count = 0
    calendar_loss = torch.tensor(0.)
    butterfly_loss = torch.tensor(0.)
    for elem in X_train_scaled:
        y= model.forward(elem)
        dydx = grad(y, elem, create_graph = True)[0]
        dydT = dydx[T_loc]
        if dydT < 0.0:
            calendar_arbi_count += 1
            calendar_loss += torch.exp(-dydT) * 1e-1
    loss += calendar_loss
    ## butterfly arbitrage
    butterfly_count = 0
    dydk = dydx[k_loc] # dCdk w.r.t. log-strike 
    d2ydk2 = grad(dydx[k_loc],elem, create_graph = True)[0][k_loc] # d2Cdk2 w.r.t. log-strike
    # butterfly arbitrage: d2CdK2 = 1/K^2(d2Cdk2 - dCdk) > 0, d2Cdk2 > dCdk
    # dCdK = 1/K*dcdk
    # d2CdK2 = 1/e^2k * (d2Cdk2 - dCdk)
    butter_ineq = (d2ydk2 - dydk)/torch.exp(2.0*elem[k_loc])
    if butter_ineq < 0.0: # violation of butterfly arbitrage
        butterfly_count += 1
        butterfly_loss += torch.exp(-butter_ineq) * 1e-1    
    loss += butterfly_loss

    # regularizations
    for param in model.parameters():
        l2_reg += torch.norm(param)
        loss += l2_lambda * l2_reg
    loss_arr.append(loss)
    if calendar_arbi_count > 0: print('calendar ',i, ' ',calendar_arbi_count,' ',calendar_loss)
    if butterfly_count > 0: print('butterfly ',i, ' ',butterfly_count,' ',butterfly_loss)
 
    if i % 500 == 0:
        print(f'Epoch: {i} Loss: {loss}')
 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
if save_net: torch.save(model.state_dict(), "heston_NN") # save the model
"""